### Importação de configurações e funções

In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *
from Utils.bronze_csv_loader import *

apply_storage_config(spark)

In [0]:
from Utils.incremental_bronze import *
from pyspark.sql.functions import (
    input_file_name,
    current_timestamp,
    lit,
    col,
    count,
    year,
    month,
    concat,
    regexp_extract
)
from pyspark.sql.types import StructType, StructField, StringType
import re, os, zipfile

### Criação da tabela de controle de arquivos

In [0]:
def criar_tabela_controle_cnpj():
    """
    Cria tabela para rastrear arquivos CNPJ já processados
    """
    spark.sql("""
        CREATE TABLE IF NOT EXISTS bronze.arquivos_processados_cnpj (
            caminho_arquivo STRING,
            nome_arquivo STRING,
            tipo_operacao STRING,
            data_processamento TIMESTAMP,
            num_registros BIGINT
        )
        USING DELTA
    """)
    print("✔ Tabela de controle de arquivos CNPJ criada!")

criar_tabela_controle_cnpj()

### Descompactação de arquivos ZIP

In [0]:
import shutil

input_path = f"{cnpj_path}"
output_path = "/dbfs/tmp/deszipados"

# Cria a pasta de destino se não existir
os.makedirs(output_path, exist_ok=True)

def decompress_file(arquivo):
    """
    Descompacta um arquivo zip do DBFS e consolida arquivos por tipo
    Empresas1, Empresas2, Empresas3 → Empresas/
    """
    if arquivo.name.endswith(".zip"):
        try:
            # Caminho do zip no DBFS
            caminho_zip_dbfs = arquivo.path

            # Caminho local temporário para descompactar
            caminho_zip_local = f"/tmp/{arquivo.name}"

            # Copia do DBFS para local
            dbutils.fs.cp(caminho_zip_dbfs, f"file:{caminho_zip_local}")

            # Cria uma subpasta temporária para descompactar
            nome_subpasta = os.path.splitext(arquivo.name)[0]
            caminho_temp = os.path.join("/tmp", f"temp_{nome_subpasta}")
            os.makedirs(caminho_temp, exist_ok=True)

            # Descompacta na pasta temporária
            with zipfile.ZipFile(caminho_zip_local, 'r') as zip_ref:
                zip_ref.extractall(caminho_temp)
            
            # Identificar o tipo base (Empresas, Estabelecimentos, Socios, etc.)
            # Remove números do final: Empresas1 → Empresas
            tipo_base = ''.join([c for c in nome_subpasta if not c.isdigit()])
            
            # Criar pasta consolidada
            caminho_consolidado = os.path.join(output_path, tipo_base)
            os.makedirs(caminho_consolidado, exist_ok=True)
            
            # Copiar todos os arquivos para a pasta consolidada
            arquivos_copiados = 0
            for root, dirs, files in os.walk(caminho_temp):
                for file in files:
                    origem = os.path.join(root, file)
                    destino = os.path.join(caminho_consolidado, file)
                    
                    # Se arquivo já existe, renomear para evitar sobrescrita
                    if os.path.exists(destino):
                        base, ext = os.path.splitext(file)
                        contador = 1
                        while os.path.exists(destino):
                            destino = os.path.join(caminho_consolidado, f"{base}_{contador}{ext}")
                            contador += 1
                    
                    # Usar shutil.copy ao invés de os.rename (cross-device)
                    shutil.copy2(origem, destino)
                    arquivos_copiados += 1
            
            print(f"Descompactado: {arquivo.name} → {tipo_base}/ ({arquivos_copiados} arquivo(s))")
            
            # Limpeza
            os.remove(caminho_zip_local)
            
            # Remover pasta temporária
            shutil.rmtree(caminho_temp)
            
            return True
            
        except Exception as e:
            print(f"Erro ao descompactar {arquivo.name}: {e}")
            return False

# Lista arquivos na pasta do DBFS
arquivos = dbutils.fs.ls(input_path)

# Descomprime cada arquivo
total_descompactados = sum([1 for arquivo in arquivos if decompress_file(arquivo)])

print(f"\n{'=' * 60}")
print(f"✔ Total de ZIPs descompactados: {total_descompactados}")
print(f"{'=' * 60}")


### Atribuição de variáveis e paths

In [0]:
TIPO_EMPRESAS = "EMPRESAS"
TIPO_ESTABELECIMENTOS = "ESTABELECIMENTOS"
TIPO_SIMPLES = "SIMPLES"
TIPO_SOCIOS = "SOCIOS"
TIPO_PAISES = "PAISES"
TIPO_MUNICIPIOS = "MUNICIPIOS"
TIPO_QUALIFICACOES = "QUALIFICACOES"
TIPO_NATUREZAS = "NATUREZAS"
TIPO_CNAES = "CNAES"
TIPO_MOTIVOS = "MOTIVOS"

# CORRIGIDO: Usar caminho DBFS
input_path_descompactado = "dbfs:/tmp/deszipados"  # ← MUDANÇA AQUI

output_path_empresas = f"{bronze_path}cnpj/empresas"
output_path_estabelecimentos = f"{bronze_path}cnpj/estabelecimentos"
output_path_simples = f"{bronze_path}cnpj/simples"
output_path_socios = f"{bronze_path}cnpj/socios"
output_path_paises = f"{bronze_path}cnpj/paises"
output_path_municipios = f"{bronze_path}cnpj/municipios"
output_path_qualificacoes = f"{bronze_path}cnpj/qualificacoes"
output_path_naturezas = f"{bronze_path}cnpj/naturezas"
output_path_cnaes = f"{bronze_path}cnpj/cnaes"
output_path_motivos = f"{bronze_path}cnpj/motivos"

print(f"Input path: {input_path_descompactado}")
print(f"Output EMPRESAS: {output_path_empresas}")

### Ingestão EMPRESAS

In [0]:
print("=" * 60)
print("INICIANDO INGESTÃO EMPRESAS")
print("=" * 60)

empresas_columns = [
    "CNPJ_BASICO",
    "RAZAO_SOCIAL_NOME_EMPRESARIAL",
    "NATUREZA_JURIDICA",
    "QUALIFICACAO_DO_RESPONSAVEL",
    "CAPITAL_SOCIAL_DA_EMPRESA",
    "PORTE_DA_EMPRESA",
    "ENTE_FEDERATIVO_RESPONSAVEL"
]

schema_empresas = StructType([StructField(c, StringType(), True) for c in empresas_columns])

# CORRIGIDO: input_path direto + sem glob
df_raw_empresas, df_empresas_corrupt, empresas_cols = read_csv_with_quotes(
    spark=spark,
    input_path="dbfs:/tmp/deszipados/Empresas/*.EMPRECSV",  # ← Caminho completo
    delimiter=";",
    encoding="iso-8859-1",
    recursive=False,  # ← Não recursivo
    path_glob_filter=None,  # ← Sem filtro
    header=False,
    schema=schema_empresas,
    expected_cols=None,
    multiline=True,
    quote="\"",
    escape="\"",
    mode="PERMISSIVE",
    corrupt_col="_corrupt_record",
    ignore_leading_trailing_ws=True,
    quarantine_path="/mnt/bronze_quarentena/cnpj/empresas",
    quarantine_mode="append"
)

print(f"EMPRESAS lidos: {df_raw_empresas.count()} registros")
print(f"EMPRESAS corrompidos: {df_empresas_corrupt.count() if df_empresas_corrupt is not None else 0} registros")


In [0]:
# Aplicar incremental APÓS leitura
try:
    df_processados_empresas = spark.table("bronze.arquivos_processados_cnpj") \
        .filter(col("tipo_operacao") == TIPO_EMPRESAS) \
        .select("nome_arquivo") \
        .distinct()
except:
    df_processados_empresas = spark.createDataFrame([], StructType([StructField("nome_arquivo", StringType(), True)]))

df_novos_empresas = (
    df_raw_empresas
    .withColumn("origin_file_name", input_file_name())
    .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
)

# Filtro incremental por nome
df_novos_empresas = df_novos_empresas.join(
    df_processados_empresas,
    df_novos_empresas.file_name_only == df_processados_empresas.nome_arquivo,
    "left_anti"
)

print(f"EMPRESAS novos para processar: {df_novos_empresas.count()}")

In [0]:
# Enriquecimento Bronze
df_bronze_empresas = (
    df_novos_empresas
    .withColumn("ingestion_dt", current_timestamp())
    .withColumn("anomes", concat(year(col("ingestion_dt")), month(col("ingestion_dt"))))
)

# Escrita incremental (append)
(
    df_bronze_empresas
    .write
    .format("delta")
    .mode("append")
    .partitionBy("anomes")
    .save(output_path_empresas)
)

print(f"EMPRESAS escritos em Bronze: {df_bronze_empresas.count()} registros")

In [0]:
# Registrar arquivos processados (apenas os novos)
df_registro_empresas = (
    df_bronze_empresas
    .groupBy("origin_file_name", "file_name_only")
    .agg(count("*").alias("num_registros"))
    .select(
        col("origin_file_name").alias("caminho_arquivo"),
        col("file_name_only").alias("nome_arquivo"),
        lit(TIPO_EMPRESAS).alias("tipo_operacao"),
        current_timestamp().alias("data_processamento"),
        col("num_registros")
    )
)

# Escrita na tabela de controle
(
    df_registro_empresas
    .write
    .format("delta")
    .mode("append")
    .saveAsTable("bronze.arquivos_processados_cnpj")
)

print("EMPRESAS registrados em controle")

In [0]:
%sql
SELECT * FROM bronze.arquivos_processados_cnpj

### Ingestão ESTABELECIMENTOS

In [0]:
print("=" * 60)
print("INICIANDO INGESTÃO ESTABELECIMENTOS")
print("=" * 60)

estabelecimentos_columns = [
    "CNPJ_BASICO",
    "CNPJ_ORDEM",
    "CNPJ_DV",
    "IDENTIFICADOR_MATRIZ_FILIAL",
    "NOME_FANTASIA",
    "SITUACAO_CADASTRAL",
    "DATA_SITUACAO_CADASTRAL",
    "MOTIVO_SITUACAO_CADASTRAL",
    "NOME_CIDADE_EXTERIOR",
    "PAIS",
    "DT_INICIO_ATIVIDADE",
    "CNAE_FISCAL_PRINCIPAL",
    "CNAE_FISCAL_SECUNDARIA",
    "TIPO_LOGRADOURO",
    "LOGRADOURO",
    "NUMERO",
    "COMPLEMENTO",
    "BAIRRO",
    "CEP",
    "UF",
    "MUNICIPIO",
    "DDD1",
    "TELEFONE1",
    "DDD2",
    "TELEFONE2",
    "DDD_FAX",
    "FAX",
    "CORREIO_ELETRONICO",
    "SITUACAO_ESPECIAL",
    "DATA_SITUACAO_ESPECIAL"
]

schema_estabelecimentos = StructType([StructField(c, StringType(), True) for c in estabelecimentos_columns])

df_raw_estab, df_estab_corrupt, estab_cols = read_csv_with_quotes(
    spark=spark,
    input_path="dbfs:/tmp/deszipados/Estabelecimentos/*.ESTABELE",  # ← Caminho completo
    delimiter=";",
    encoding="iso-8859-1",
    recursive=False,
    path_glob_filter=None,
    header=False,
    schema=schema_estabelecimentos,
    expected_cols=None,
    multiline=True,
    quote="\"",
    escape="\"",
    mode="PERMISSIVE",
    corrupt_col="_corrupt_record",
    ignore_leading_trailing_ws=True,
    quarantine_path="/mnt/bronze_quarentena/cnpj/estabelecimentos",
    quarantine_mode="append"
)

print(f"ESTABELECIMENTOS lidos: {df_raw_estab.count()} registros")

In [0]:
try:
    df_processados_estab = spark.table("bronze.arquivos_processados_cnpj") \
        .filter(col("tipo_operacao") == "ESTABELECIMENTOS") \
        .select("nome_arquivo") \
        .distinct()
except:
    df_processados_estab = spark.createDataFrame([], StructType([StructField("nome_arquivo", StringType(), True)]))

df_novos_estab = (
    df_raw_estab
    .withColumn("origin_file_name", input_file_name())
    .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
)

df_novos_estab = df_novos_estab.join(
    df_processados_estab,
    df_novos_estab.file_name_only == df_processados_estab.nome_arquivo,
    "left_anti"
)

print(f"ESTABELECIMENTOS novos para processar: {df_novos_estab.count()}")

In [0]:
# Enriquecimento Bronze
df_bronze_estab = (
    df_novos_estab
    .withColumn("ingestion_dt", current_timestamp())
    .withColumn("anomes", concat(year(col("ingestion_dt")), month(col("ingestion_dt"))))
)

(
    df_bronze_estab
    .write
    .format("delta")
    .mode("append")
    .partitionBy("anomes")
    .save(f"{bronze_path}cnpj/estabelecimentos")
)

print(f"ESTABELECIMENTOS escritos em Bronze: {df_bronze_estab.count()} registros")

In [0]:
# Registrar arquivos processados
df_registro_estab = (
    df_bronze_estab
    .groupBy("origin_file_name", "file_name_only")
    .agg(count("*").alias("num_registros"))
    .select(
        col("origin_file_name").alias("caminho_arquivo"),
        col("file_name_only").alias("nome_arquivo"),
        lit("ESTABELECIMENTOS").alias("tipo_operacao"),
        current_timestamp().alias("data_processamento"),
        col("num_registros")
    )
)

(
    df_registro_estab
    .write
    .format("delta")
    .mode("append")
    .saveAsTable("bronze.arquivos_processados_cnpj")
)

print("ESTABELECIMENTOS registrados em controle")

### Ingestão SIMPLES

In [0]:
print("=" * 60)
print("INICIANDO INGESTÃO SIMPLES")
print("=" * 60)

simples_columns = [
    "CNPJ_BASICO",
    "OPCAO_PELO_SIMPLES",
    "DATA_DE_OPCAO_PELO_SIMPLES",
    "DATA_DE_EXCLUSAO_DO_SIMPLES",
    "OPCAO_PELO_MEI",
    "DATA_DE_OPCAO_PELO_MEI",
    "DATA_DE_EXCLUSAO_DO_MEI"
]

schema_simples = StructType([StructField(c, StringType(), True) for c in simples_columns])

df_raw_simples, df_simples_corrupt, simples_cols = read_csv_with_quotes(
    spark=spark,
    input_path='dbfs:/tmp/deszipados/Simples',
    delimiter=";",
    encoding="iso-8859-1",
    recursive=True,
    path_glob_filter=None,
    header=False,
    schema=schema_simples,
    expected_cols=None,
    multiline=True,
    quote="\"",
    escape="\"",
    mode="PERMISSIVE",
    corrupt_col="_corrupt_record",
    ignore_leading_trailing_ws=True,
    quarantine_path="/mnt/bronze_quarentena/cnpj/simples",
    quarantine_mode="append"
)

print(f"SIMPLES lidos: {df_raw_simples.count()} registros")

In [0]:
try:
    df_processados_simples = spark.table("bronze.arquivos_processados_cnpj") \
        .filter(col("tipo_operacao") == TIPO_SIMPLES) \
        .select("nome_arquivo") \
        .distinct()
except:
    df_processados_simples = spark.createDataFrame([], StructType([StructField("nome_arquivo", StringType(), True)]))

df_novos_simples = (
    df_raw_simples
    .withColumn("origin_file_name", input_file_name())
    .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
)

df_novos_simples = df_novos_simples.join(
    df_processados_simples,
    df_novos_simples.file_name_only == df_processados_simples.nome_arquivo,
    "left_anti"
)

print(f"SIMPLES novos para processar: {df_novos_simples.count()}")

In [0]:
df_bronze_simples = (
    df_novos_simples
    .withColumn("ingestion_dt", current_timestamp())
)

(
    df_bronze_simples
    .write
    .format("delta")
    .mode("append")
    .save(output_path_simples)
)

print(f"SIMPLES escritos em Bronze: {df_bronze_simples.count()} registros")

In [0]:
df_registro_simples = (
    df_bronze_simples
    .groupBy("origin_file_name", "file_name_only")
    .agg(count("*").alias("num_registros"))
    .select(
        col("origin_file_name").alias("caminho_arquivo"),
        col("file_name_only").alias("nome_arquivo"),
        lit(TIPO_SIMPLES).alias("tipo_operacao"),
        current_timestamp().alias("data_processamento"),
        col("num_registros")
    )
)

(
    df_registro_simples
    .write
    .format("delta")
    .mode("append")
    .saveAsTable("bronze.arquivos_processados_cnpj")
)

print("SIMPLES registrados em controle")

### Ingestão SOCIOS

In [0]:
print("=" * 60)
print("INICIANDO INGESTÃO SOCIOS")
print("=" * 60)

socios_columns = [
    "CNPJ_BASICO",
    "IDENTIFICADOR_DO_SOCIO",
    "NOME_DO_SOCIO",
    "CPF_CNPJ_DO_SOCIO",
    "QUALIFICACAO_DO_SOCIO",
    "DATA_DE_ENTRADA_NA_SOCIEDADE",
    "PAIS",
    "REPRESENTANTE_LEGAL",
    "NOME_DO_REPRESENTANTE_LEGAL",
    "QUALIFICACAO_DO_REPRESENTANTE_LEGAL",
    "FAIXA_ETARIA"
]

schema_socios = StructType([StructField(c, StringType(), True) for c in socios_columns])

df_raw_socios, df_socios_corrupt, socios_cols = read_csv_with_quotes(
    spark=spark,
    input_path='dbfs:/tmp/deszipados/Socios',
    delimiter=";",
    encoding="iso-8859-1",
    recursive=True,
    path_glob_filter=None,
    header=False,
    schema=schema_socios,
    expected_cols=None,
    multiline=True,
    quote="\"",
    escape="\"",
    mode="PERMISSIVE",
    corrupt_col="_corrupt_record",
    ignore_leading_trailing_ws=True,
    quarantine_path="/mnt/bronze_quarentena/cnpj/socios",
    quarantine_mode="append"
)

print(f"✔ SOCIOS lidos: {df_raw_socios.count()} registros")

In [0]:
try:
    df_processados_socios = spark.table("bronze.arquivos_processados_cnpj") \
        .filter(col("tipo_operacao") == TIPO_SOCIOS) \
        .select("nome_arquivo") \
        .distinct()
except:
    df_processados_socios = spark.createDataFrame([], StructType([StructField("nome_arquivo", StringType(), True)]))

df_novos_socios = (
    df_raw_socios
    .withColumn("origin_file_name", input_file_name())
    .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
)

df_novos_socios = df_novos_socios.join(
    df_processados_socios,
    df_novos_socios.file_name_only == df_processados_socios.nome_arquivo,
    "left_anti"
)

print(f"SOCIOS novos para processar: {df_novos_socios.count()}")

In [0]:
df_bronze_socios = (
    df_novos_socios
    .withColumn("ingestion_dt", current_timestamp())
)

(
    df_bronze_socios
    .write
    .format("delta")
    .mode("append")
    .save(output_path_socios)
)

print(f"SOCIOS escritos em Bronze: {df_bronze_socios.count()} registros")

In [0]:
df_registro_socios = (
    df_bronze_socios
    .groupBy("origin_file_name", "file_name_only")
    .agg(count("*").alias("num_registros"))
    .select(
        col("origin_file_name").alias("caminho_arquivo"),
        col("file_name_only").alias("nome_arquivo"),
        lit(TIPO_SOCIOS).alias("tipo_operacao"),
        current_timestamp().alias("data_processamento"),
        col("num_registros")
    )
)

(
    df_registro_socios
    .write
    .format("delta")
    .mode("append")
    .saveAsTable("bronze.arquivos_processados_cnpj")
)

print("SOCIOS registrados em controle")

### Ingestão TABELAS AUXILIARES

In [0]:
tabelas_auxiliares = [
    ("Paises", "PAISES", output_path_paises, "*"),
    ("Municipios", "MUNICIPIOS", output_path_municipios, "*"),
    ("Qualificacoes", "QUALIFICACOES", output_path_qualificacoes, "*"),
    ("Naturezas", "NATUREZAS", output_path_naturezas, "*"),
    ("Cnaes", "CNAES", output_path_cnaes, "*"),
    ("Motivos", "MOTIVOS", output_path_motivos, "*")
]

aux_columns = ["CODIGO", "DESCRICAO"]
schema_aux = StructType([StructField(c, StringType(), True) for c in aux_columns])

for pasta, tipo_operacao, output_path, extensao in tabelas_auxiliares:
    
    print("=" * 60)
    print(f"INICIANDO INGESTÃO {tipo_operacao}")
    print("=" * 60)
    
    try:
        # CORRIGIDO: Usar caminho completo direto (Teste 1)
        input_path_completo = f"dbfs:/tmp/deszipados/{pasta}/{extensao}"
        
        print(f"Lendo de: {input_path_completo}")
        
        # Leitura com read_csv_with_quotes
        df_raw_aux, df_aux_corrupt, aux_cols = read_csv_with_quotes(
            spark=spark,
            input_path=input_path_completo,  # ← Caminho completo
            delimiter=";",
            encoding="iso-8859-1",
            recursive=False,  # ← Não recursivo
            path_glob_filter=None,  # ← Sem filtro
            header=False,
            schema=schema_aux,
            expected_cols=None,
            multiline=True,
            quote="\"",
            escape="\"",
            mode="PERMISSIVE",
            corrupt_col="_corrupt_record",
            ignore_leading_trailing_ws=True,
            quarantine_path=f"/mnt/bronze_quarentena/cnpj/{pasta.lower()}",
            quarantine_mode="append"
        )
        
        print(f"{tipo_operacao} lidos: {df_raw_aux.count()} registros")
        
        # Aplicar incremental
        try:
            df_processados_aux = spark.table("bronze.arquivos_processados_cnpj") \
                .filter(col("tipo_operacao") == tipo_operacao) \
                .select("nome_arquivo") \
                .distinct()
        except:
            df_processados_aux = spark.createDataFrame([], StructType([StructField("nome_arquivo", StringType(), True)]))
        
        df_novos_aux = (
            df_raw_aux
            .withColumn("origin_file_name", input_file_name())
            .withColumn("file_name_only", regexp_extract(col("origin_file_name"), "[^/]+$", 0))
        )
        
        df_novos_aux = df_novos_aux.join(
            df_processados_aux,
            df_novos_aux.file_name_only == df_processados_aux.nome_arquivo,
            "left_anti"
        )
        
        print(f"{tipo_operacao} novos para processar: {df_novos_aux.count()}")
        
        # Enriquecimento
        df_bronze_aux = (
            df_novos_aux
            .withColumn("ingestion_dt", current_timestamp())
        )
        
        # Escrita
        (
            df_bronze_aux
            .write
            .format("delta")
            .mode("append")
            .save(output_path)
        )
        
        print(f"✔ {tipo_operacao} escritos em Bronze: {df_bronze_aux.count()} registros")
        
        # Registrar
        df_registro_aux = (
            df_bronze_aux
            .groupBy("origin_file_name", "file_name_only")
            .agg(count("*").alias("num_registros"))
            .select(
                col("origin_file_name").alias("caminho_arquivo"),
                col("file_name_only").alias("nome_arquivo"),
                lit(tipo_operacao).alias("tipo_operacao"),
                current_timestamp().alias("data_processamento"),
                col("num_registros")
            )
        )
        
        (
            df_registro_aux
            .write
            .format("delta")
            .mode("append")
            .saveAsTable("bronze.arquivos_processados_cnpj")
        )
        
        print(f"{tipo_operacao} registrados em controle\n")
        
    except Exception as e:
        print(f"ERRO ao processar {tipo_operacao}: {e}\n")

### Resumo da ingestão

In [0]:
print("=" * 60)
print("RESUMO DA INGESTÃO INCREMENTAL CNPJ")
print("=" * 60)

df_resumo = spark.sql("""
    SELECT
        tipo_operacao,
        COUNT(*) as total_arquivos,
        SUM(num_registros) as total_registros,
        MAX(data_processamento) as ultima_ingestao
    FROM bronze.arquivos_processados_cnpj
    GROUP BY tipo_operacao
    ORDER BY tipo_operacao
""")

df_resumo.display()

print("\n Ingestão CNPJ finalizada com sucesso!")